# Manipulação e Limpeza dos dados 

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm

tqdm.pandas()

### Carregar Datasets

Neste notebook, três datasets foram carregados e manipulados para análise. Os datasets em questão são:

- **offers.json**: contém informações sobre as ofertas realizadas.
- **profile.json**: contém dados do perfil dos usuários.
- **transactions.json**: contém informações sobre transações realizadas pelos usuários.

In [3]:
offers = pd.read_json("case_data/offers.json")

A seguir, foram feitas as seguintes modificações no dataset de perfil (`profile`):

1. **Substituição de valores inválidos na coluna `age`**:
    - O valor `118` na coluna `age` foi substituído por `NaN`, pois provavelmente representa um erro de digitação ou dado inválido.

2. **Conversão da coluna `registered_on` para formato de data**:
    - A coluna `registered_on`, que está em formato numérico (`YYYYMMDD`), foi convertida para o formato de data usando o método `pd.to_datetime()`.
    - Qualquer erro de conversão foi tratado com `errors='coerce'`, que substitui as datas inválidas por `NaN`.

3. **Criação da coluna `days_since_registration`**:
    - Foi calculado o número de dias desde o registro de cada usuário, com base na data mais recente de registro.

In [4]:
profile = pd.read_json("case_data/profile.json")
profile["age"] = profile.age.replace(118, np.nan)
profile["registered_on"] = pd.to_datetime(
    profile["registered_on"], format="%Y%m%d", errors="coerce"
)
max_date = profile["registered_on"].max()
profile["days_since_registration"] = (max_date - profile["registered_on"]).dt.days

No dataset de transações (`transactions`), as seguintes transformações foram realizadas:

1. **Ordenação e reset do índice**:
    - As transações foram ordenadas por `account_id` e `time_since_test_start` para garantir que os dados estivessem na sequência correta.
    - O índice foi resetado para garantir uma sequência contínua após a ordenação.

2. **Normalização da coluna `value`**:
    - A coluna `value`, que contém dados aninhados em formato JSON, foi normalizada usando `pd.json_normalize()` para expandir esses dados em colunas individuais.
    - Após isso, as colunas da transação original, exceto a coluna `value`, foram concatenadas com as novas colunas normalizadas.


3. **Tratamento na coluna `offer_id`**:
    - Valores ausentes na coluna `offer_id` foram preenchidos com a coluna `offer id`, garantindo que não houvesse valores `NaN` nas ofertas.

In [5]:
transactions = (
    pd.read_json("case_data/transactions.json")
    .sort_values(by=["account_id", "time_since_test_start"])
    .reset_index(drop=True)
    .pipe(
        lambda df: pd.concat(
            [df.drop(columns=["value"]), pd.json_normalize(df["value"])], axis=1
        )
    )
    .assign(offer_id=lambda df: df["offer id"].fillna(df["offer_id"]))
    .drop(columns=["offer id"])
)

Após as manipulações nos datasets individuais, os dados foram combinados em um único DataFrame. O objetivo foi integrar informações sobre transações, ofertas e perfis dos usuários. 

In [6]:
df = transactions.merge(offers, how="left", left_on="offer_id", right_on="id").merge(
    profile, how="left", left_on="account_id", right_on="id"
)
profile_cols = ["age", "days_since_registration", "gender", "credit_card_limit"]
offer_cols = ["offer_type", "duration", "discount_value"]
transaction_cols = [
    "event",
    "account_id",
    "time_since_test_start",
    "amount",
    "offer_id",
]
df = df[transaction_cols + profile_cols + offer_cols]

O dataset unificado é mostrado abaixo para um account_id de exemplo. Considera-se que há uma oferta aplicada numa transação quando há uma oferta completada no mesmo `time_since_test_start` transação. 

In [7]:
df.query("account_id == 'b008527a63914a51bf97fc0b800e7fe1'")

,event,account_id,time_since_test_start,amount,offer_id,age,days_since_registration,gender,credit_card_limit,offer_type,duration,discount_value
212153,offer received,b008527a63914a51bf97fc0b800e7fe1,0.00,NaN,5a8bc65990b245e5a138643cd4eb9837,23.0,1616,M,32000.0,informational,3.0,0.0
212154,offer viewed,b008527a63914a51bf97fc0b800e7fe1,3.75,NaN,5a8bc65990b245e5a138643cd4eb9837,23.0,1616,M,32000.0,informational,3.0,0.0
212155,offer received,b008527a63914a51bf97fc0b800e7fe1,7.00,NaN,2906b810c7d4411798c6938adc9daaa5,23.0,1616,M,32000.0,discount,7.0,2.0
212156,transaction,b008527a63914a51bf97fc0b800e7fe1,8.00,0.20,None,23.0,1616,M,32000.0,NaN,NaN,NaN
212157,transaction,b008527a63914a51bf97fc0b800e7fe1,8.25,1.42,None,23.0,1616,M,32000.0,NaN,NaN,NaN
212158,offer viewed,b008527a63914a51bf97fc0b800e7fe1,8.75,NaN,2906b810c7d4411798c6938adc9daaa5,23.0,1616,M,32000.0,discount,7.0,2.0
212159,transaction,b008527a63914a51bf97fc0b800e7fe1,8.75,1.76,None,23.0,1616,M,32000.0,NaN,NaN,NaN
212160,transaction,b008527a63914a51bf97fc0b800e7fe1,9.00,2.38,None,23.0,1616,M,32000.0,NaN,NaN,NaN
212161,transaction,b008527a63914a51bf97fc0b800e7fe1,9.25,1.69,None,23.0,1616,M,32000.0,NaN,NaN,NaN
212162,transaction,b008527a63914a51bf97fc0b800e7fe1,10.50,1.31,None,23.0,1616,M,32000.0,NaN,NaN,NaN


Após a manipulação e combinação dos dados, o DataFrame final foi salvo no formato Parquet utilizando o método to_parquet. Esse formato é eficiente para armazenamento e leitura de grandes volumes de dados

In [8]:
df.to_parquet("files/data_cleaned.parquet", index=False)